In [1]:
import pandas as pd
from sqlalchemy import create_engine
import requests

# Consultas de Metadatos: PostgreSQL

In [2]:
usuario = "userPSQL"
contraseña = "passPSQL"
host = "localhost"  
puerto = "5432"
base_datos = "postgres"

engine = create_engine(f"postgresql+psycopg2://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")

Devolver toda la información de todas las asignaturas

In [3]:
query = """SELECT * FROM ASIGNATURAS"""
df = pd.read_sql(query, engine)
df.head()

,id,nombre,numero_creditos,agno_academico,semestre,idioma
0,615001039,Proyecto de Ciencia de Datos,6,Cuarto curso,Séptimo semestre,Castellano
1,615000727,Traductores de Lenguajes de Programacion,6,Tercero curso,Sexto semestre,Castellano
2,615000720,Programacion Concurrente y Avanzada,6,Segundo curso,Cuarto semestre,Castellano
3,615000335,Sistemas Basados en Computador,6,Cuarto curso,Séptimo semestre,Castellano
4,615000222,Taller de Programacion,3,Primer curso,Primer semestre,Castellano


Devolver todos los profesores que imparten la asignatura Robótica

In [4]:
query = """
       SELECT P.* 
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.id
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       WHERE A.NOMBRE = 'Robótica';
    """
df = pd.read_sql(query, engine)
df.head()

,id,nombre,correo_electronico
0,104,Jose Eugenio Naranjo,joseeugenio.naranjo@upm.e\ns
1,47,Alfredo Valle Barrio,alfredo.valle@upm.es


Devolver los profesores que impartan más de 3 asignaturas

In [5]:
query = """
       SELECT P.NOMBRE, COUNT(A.ID) AS NUM_ASIGNATURAS
       FROM PROFESORES P 
       JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
       JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
       GROUP BY P.NOMBRE
       HAVING COUNT(A.ID) > 3;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Edgar Talavera Muñoz,5
1,Victor Jose Martinez,6
2,Sergio Alejandro D'antonio,4
3,Joaquin Entrialgo Castaño,5
4,Julio Cesar Hernandez,4


Devolver todas el profesor que imparte el mayor numero de asignaturas de la titulación Grado en Sistemas de Informacion

In [6]:
query = """
        SELECT P.NOMBRE,COUNT(DISTINCT A.ID) AS NUM_ASIGNATURAS
        FROM PROFESORES P
        JOIN PROFESORESASIGNATURAS AP ON AP.PROFESOR_ID = P.ID
        JOIN ASIGNATURAS A ON A.ID = AP.ASIGNATURA_ID
        JOIN TITULACIONESASIGNATURAS TA ON TA.ASIGNATURA_ID = A.ID
        JOIN TITULACIONES T ON T.ID = TA.TITULACION_ID
        WHERE T.NOMBRE = 'Grado en Sistemas de Informacion'
        GROUP BY  P.NOMBRE
        ORDER BY NUM_ASIGNATURAS DESC
        LIMIT 1;
    """
df = pd.read_sql(query, engine)
df.head()

,nombre,num_asignaturas
0,Agustin Yague Panadero,4


# Consultas de Metaesquema: GraphDB (SPARQL)

In [8]:
GRAPHDB_ENDPOINT = "http://localhost:8000/repositories/asignaturas"

## Consulta 1
Todas las asignaturas impartidas por una escuela

In [ ]:
query = """
PREFIX upm: <http://upm.es/ontology/>
SELECT ?nombre_escuela ?nombre_asignatura
WHERE {
    ?s a upm:Escuela .
    ?s upm:nombre ?nombre_escuela .
    ?s upm:imparteTitulacion ?t .
    ?t upm:incluyeAsignatura ?a .
    ?a upm:nombre ?nombre_asignatura .
} LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_escuela = r.get('nombre_escuela', {}).get('value', '')
        nombre_asignatura = r.get('nombre_asignatura', {}).get('value', '')
        print(f"Escuela: {nombre_escuela}, Asignatura: {nombre_asignatura}")
else:
    print("Error:", response.status_code, response.text)

SyntaxError: incomplete input (2778895369.py, line 26)

## Consulta 2
Encontrar la escuela a la que pertenece el profesor de una asignatura

In [ ]:
query = """
PREFIX upm: <http://upm.es/ontology/>
SELECT ?nombre_escuela ?nombre_profesor
WHERE {
    ?e a upm:Escuela .
    ?e upm:nombre ?nombre_escuela .
    ?e upm:imparteTitulacion ?t .
    ?t upm:incluyeAsignatura ?a .
    ?a upm:tieneProfesor ?p .
    ?p upm:nombre ?nombre_profesor .
} LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_escuela = r.get('nombre_escuela', {}).get('value', '')
        nombre_profesor = r.get('nombre_profesor', {}).get('value', '')
        print(f"Escuela: {nombre_escuela}, Profesor: {nombre_profesor}")
else:
    print("Error:", response.status_code, response.text)

Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Cristian Oliver Ramirez
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Raul Lara Cabrera
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Andrea Jesus Cimmino
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Celena Lorenzo Navarro
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Francisco Serradilla Garcia
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Raul Lara Cabrera
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Vicente Angel Garcia
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: David Camacho Fernandez
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Jesus Garcia Lopez De
Escuela: E.T.S De Ing. De Sistemas Informáticos, Profesor: Borja Bordel Sanchez


## Consulta 3
Profesores que imparten más de N asignaturas

In [16]:
query = """
PREFIX upm: <http://upm.es/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?profesor ?nombre (COUNT(?asig) AS ?numAsignaturas) WHERE {
  ?asig rdf:type upm:Asignatura .
  ?asig upm:tieneProfesor ?profesor .
  OPTIONAL { ?profesor upm:nombre ?nombre. }
}
GROUP BY ?profesor ?nombre
HAVING (COUNT(?asig) > 4)
ORDER BY DESC(?numAsignaturas)
LIMIT 10
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        nombre_profesor = r.get('nombre', {}).get('value', '')
        num_asignaturas = r.get('numAsignaturas', {}).get('value', '')
        print(f"Profesor: {nombre_profesor}, Número de asignaturas: {num_asignaturas}")
else:
    print("Error:", response.status_code, response.text)

Profesor: Joaquin Gayoso Cabada, Número de asignaturas: 13
Profesor: Cristian Oliver Ramirez, Número de asignaturas: 10
Profesor: Carlos Castilla Ruiz, Número de asignaturas: 9
Profesor: Maria Celia Fernandez Aller, Número de asignaturas: 9
Profesor: Maria Angeles Mahillo, Número de asignaturas: 9
Profesor: Rafael Miñano Rubio, Número de asignaturas: 9
Profesor: Jordi Burguet Castell, Número de asignaturas: 8
Profesor: Agustin Yague Panadero, Número de asignaturas: 8
Profesor: Javier Huertas Tato, Número de asignaturas: 8
Profesor: Carlos Camacho Gomez, Número de asignaturas: 8


## Consulta 4
Distribución del número de creditos de cada asignatura por titulación (indicadores: media, mínimo, máximo)

In [18]:
query = """
PREFIX upm: <http://upm.es/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?titulacion ?titNombre (AVG(?c) AS ?avgCreditos) (MIN(?c) AS ?minCreditos) (MAX(?c) AS ?maxCreditos) WHERE {
  ?titulacion a upm:Titulacion .
  OPTIONAL { ?titulacion upm:nombre ?titNombre. }
  ?titulacion upm:incluyeAsignatura ?asig .
  ?asig upm:creditosECTS ?c .
}
GROUP BY ?titulacion ?titNombre
ORDER BY DESC(?avgCreditos)
"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
        titNombre = r.get('titNombre', {}).get('value', '')
        avgCreditos = r.get('avgCreditos', {}).get('value', '')
        minCreditos = r.get('minCreditos', {}).get('value', '')
        maxCreditos = r.get('maxCreditos', {}).get('value', '')
        print(f"Titulación: {titNombre}, Créditos - Promedio: {avgCreditos}, Mínimo: {minCreditos}, Máximo: {maxCreditos}")
else:
    print("Error:", response.status_code, response.text)

Titulación: Grado en Ciencia de Datos e Inteligencia Artificial, Créditos - Promedio: , Mínimo: 3, Máximo: 6
Titulación: Grado en Sistemas de Informacion, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Ingenieria del Software, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Ingenieria de Computadores, Créditos - Promedio: , Mínimo: 3, Máximo: 9
Titulación: Grado en Tecnologias para la Sociedad de la Informacion, Créditos - Promedio: , Mínimo: 3, Máximo: 9


## Consulta 5
Obtener la dirección postal, web, descripción, director y universidad a la que pertenecen las escuelas.

In [19]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?o ?nombreUni ?l ?w ?abstract ?director
WHERE {
    ?s upm:entidad_dbpedia ?o .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?o dbpe:localización ?l .
    ?o dbpe:sitioWeb ?w .
    ?o dbpe:director ?director .
    ?o dbo:abstract ?abstract .
  }
}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      o = r.get('o', {}).get('value', '')
      nombreUni = r.get('nombreUni', {}).get('value', '')
      l = r.get('l', {}).get('value', '')
      w = r.get('w', {}).get('value', '')
      abstract = r.get('abstract', {}).get('value', '')
      director = r.get('director', {}).get('value', '')

      print(f"Escuela: {o}")
      print(f"Universidad: {nombreUni}")
      print(f"Localización: {l}")
      print(f"Sitio web: {w}")
      print(f"Director: {director}")
      print(f"Abstract: {abstract}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Escuela: http://es.dbpedia.org/resource/Escuela_Técnica_Superior_de_Ingeniería_de_Sistemas_Informáticos_(Universidad_Politécnica_de_Madrid)
Universidad: Universidad Politécnica de Madrid
Localización: Calle Alan Turing s/n , 28031, Madrid,  España
Sitio web: http://www.etsisi.upm.es/
Director: Agustín Yagüe Panadero
Abstract: La Escuela Técnica Superior de Ingeniería de Sistemas Informáticos (ETSISI) es un centro de la Universidad Politécnica de Madrid que imparte las ramas de la Ingeniería Informática, en concreto las titulaciones de Grado en Ingeniería de Software, Ingeniería de Computadores, Sistemas de Información y Tecnologías para la Sociedad de la Información. La Escuela Técnica Superior de Ingeniería de Sistemas Informáticos, junto con la Escuela Técnica Superior de Ingeniería y Sistemas de Telecomunicación, la Escuela Técnica Superior de Ingenieros en Topografía, Geodesia y Cartografía, y el Centro Superior de Diseño de Moda de Madrid, integran el complejo universitario del Ca

## Consulta 6
Obtener información de la universidad

In [20]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?otra
WHERE {
    ?s upm:entidad_dbpedia ?o .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?u dbo:affiliation ?affiliation .
    ?otra dbo:affiliation ?affiliation .
  }
}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      otra = r.get('otra', {}).get('value', '')
      otra2 = r.get('otra2', {}).get('value', '')

      print(f"Asociada: {otra}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Asociada: http://es.dbpedia.org/resource/Universidad_de_Nápoles_Federico_II
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_Técnica_de_Delft
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/École_nationale_de_l'aviation_civile
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Institut_Supérieur_de_l'Aéronautique_et_de_l'Espace
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_de_Stuttgart
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Politécnico_de_Milán
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universidad_Politécnica_de_Valencia
------------------------------------------------------------
Asociada: http://es.dbpedia.org/resource/Universida

## Consulta 8
Asignaturas que imparte el director/a de la escuela.

In [22]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX dbo:  <http://dbpedia.org/ontology/>
PREFIX dbr:  <http://dbpedia.org/resource/>
PREFIX dbpe:  <http://es.dbpedia.org/property/>
PREFIX upm:  <http://upm.es/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>


SELECT ?director_norm ?n
WHERE {
    ?s upm:entidad_dbpedia ?o .
    ?a upm:tieneProfesor ?p .
    ?p upm:nombre ?localName .
    ?a upm:nombre ?n .

  SERVICE <https://es.dbpedia.org/sparql> {
    ?o dbpe:campus ?u .
    ?u dbpe:nombre ?nombreUni .
    ?o dbpe:localización ?l .
    ?o dbpe:sitioWeb ?w .
    ?o dbpe:director ?director .
    ?o dbo:abstract ?abstract .

    BIND(REPLACE(REPLACE(?director, "ü", "u"), "í", "i") AS ?director_norm)

  }

  FILTER(?localName = ?director_norm)

}

"""

response = requests.post(
    GRAPHDB_ENDPOINT,
    data={'query': query},
    headers={'Accept': 'application/sparql-results+json'}
)

if response.status_code == 200:
    results = response.json()
    for r in results['results']['bindings']:
      director = r.get('director_norm', {}).get('value', '')
      asignatura = r.get('n', {}).get('value', '')
      print(f"Director: {director}")
      print(f"Asignatura: {asignatura}")
      print("-" * 60)
else:
    print("Error:", response.status_code, response.text)

Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Metricas y Modelos para el Control y Gestion de Servicios
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Mejores Practicas para Gestion de Servicios
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Verificacion y Validacion
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria de Software
------------------------------------------------------------
Director: Agustin Yague Panadero
Asignatura: Fundamentos de Ingenieria del Software
------------------------------------